In [ ]:
from pathlib import Path
from random import choice
import cv2
import ipywidgets as widgets
from IPython.display import display
import numpy as np

In [ ]:
%run Mask-Utils.ipynb

In [ ]:

# ----------------------------- #


In [ ]:
TEXTURE_SIZE = 400

In [ ]:
FACECOVER_ROOT = 'masks/'
FACECOVER_EXT = '.png'
FACECOVER_PATHS = list(sorted(str(p) for p in Path(FACECOVER_ROOT).glob('*' + FACECOVER_EXT)))
assert len(FACECOVER_PATHS), 'Failed to find any masks in "masks" directory'

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    face_boxes, tddfa = load_models()

In [ ]:
predictor = get_vertices_predictor(face_boxes, tddfa, is_cropped=False)

In [ ]:
def get_image_with_text(text, color):
    image = np.full((333, 999, 3), dtype=np.uint8, fill_value=255) 
    image = cv2.putText(image, text, (100, 160), cv2.FONT_HERSHEY_SIMPLEX ,  
                        1, color, 1, cv2.LINE_AA)
    return image

In [ ]:
def cover_face_with_mask(
    image,
    facecover_image,
    cache=dict(
        face_indicator = None,
        convexer = None,
        blur_mix_weights = None,
        linear_nd_creator = CachedInterpolatorCreator(interpolate.LinearNDInterpolator),
        linear_nd_ext_creator = CachedInterpolatorCreator(LinearNDInterpolatorExt),
    )):
    assert image is not None

    try:
        ver = predictor(image)
    except IndexError:
        return None
        
    texture = uv_tex(image, [ver], tddfa.tri, uv_h=TEXTURE_SIZE, uv_w=TEXTURE_SIZE)

    uv_coords = get_uv_coords(TEXTURE_SIZE)
    uv2ver_sparse = get_sparse_uv2ver(uv_coords, TEXTURE_SIZE, ver)
    uv2ver = inpaint(uv2ver_sparse,
                     mask=np.isnan(uv2ver_sparse).astype(np.uint8),
                     interpolator_cls=cache['linear_nd_creator'])

    if cache['face_indicator'] is None:
        cache['face_indicator'] = get_face_indicator(uv2ver)
    if cache['convexer'] is None:
        cache['convexer'] = FixedConvexer(ver, uv_coords, cache['face_indicator'], uv2ver)
    
    convex_uv2ver_sparse = cache['convexer'](uv2ver)
    convex_uv2ver = inpaint(convex_uv2ver_sparse,
                            mask=np.isnan(convex_uv2ver_sparse).astype(np.uint8),
                            interpolator_cls=cache['linear_nd_ext_creator'])

    if cache['blur_mix_weights'] is None:
        cache['blur_mix_weights'] = get_blur_weights(convex_uv2ver.shape)[..., None]
    
    blurred_convex_uv2ver = blur_multichannel(convex_uv2ver, sigma=(2.5, 5.0), mode='nearest')
    convex_uv2ver = convex_uv2ver * (1.0 - cache['blur_mix_weights']) + blurred_convex_uv2ver * cache['blur_mix_weights']
    
    convex_mesh = get_mesh(convex_uv2ver, cache['face_indicator'], TEXTURE_SIZE, TEXTURE_SIZE)
    
    visible_skin = get_visible_skin(uv2ver)
    target_lightness_ratio = get_cheeks_lightness_ratio(texture, visible_skin)
    light_x = find_light_x_position(
        target_lightness_ratio,
        uv_coords, ver, tddfa.tri, texture, visible_skin,
        render_app, image.shape,
    )

    facecover = FacecoverTextureWarper()(facecover_image, TEXTURE_SIZE)
    facecover_tr = facecover_color_transfer(facecover, texture)
    update_colors(convex_mesh, facecover_tr)

    ambient_w, direct_w = get_lightning_params(target_lightness_ratio, light_x, facecover_tr)
    
    final_render = render_facecover(
        render_app, convex_mesh, image, light_x, ambient_w, direct_w, return_intermediate=False
    )     
    return final_render

In [ ]:

# ----------------------------- #


In [ ]:
import albumentations as alb
from albumentations.pytorch import ToTensorV2
import re
import torch
import torchvision.models as models
from cnn_finetune import make_model
from torch.nn.functional import softmax


class MaskDetector():
    def __init__(self, model_path="mask_resnet18_epochs_5.pth"):
        search_model_name = re.search(r'.*mask_(.*)_epochs.*', model_path, flags=0)
        assert search_model_name
        model_name = search_model_name.group(1)
        model = make_model(
            model_name,
            pretrained=False,
            num_classes=2,
            dropout_p=0.2,
            input_size=(64, 64) if model_name.startswith(('vgg', 'squeezenet')) else None,
        )
        model.load_state_dict(torch.load(model_path))
        model.eval()
        
        self._model = model
        self._transform = alb.Compose([
            alb.Resize(64, 64),
            alb.Normalize(),
            ToTensorV2(),
        ])

    def __call__(self, image):
        image = self._transform(image=image)['image']
        return softmax(self._model(image[None, :]), dim=-1).detach().numpy().flatten()[1]

In [ ]:
mask_detector = MaskDetector()

In [ ]:
def classify(image_bgr):
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    score = mask_detector(image_rgb)
    return score

In [ ]:

# ----------------------------- #


In [ ]:
from io import BytesIO
from PIL import Image

def img_to_value(img):
    if img.ndim == 3:
        img = img[..., ::-1]
    buffer = BytesIO()
    Image.fromarray(img).save(buffer, 'jpeg')
    return buffer.getvalue()

## Input image

In [ ]:
upload = widgets.FileUpload(
    accept='.jpg,.jpeg,.JPG,.JPEG,.png,.PNG',
    multiple=False,
    description='Upload an image with face',
    layout=widgets.Layout(width='75%'),
)
upload

In [ ]:
def get_input_image():
    if len(upload.value) == 0:
        return None

    input_buffer = next(iter(upload.value.values()))['content']
    return cv2.imdecode(np.frombuffer(input_buffer, np.uint8), cv2.IMREAD_COLOR)

In [ ]:
input_image_preview = widgets.Image(
    value=img_to_value(np.full((10, 10, 3), dtype=np.uint8, fill_value=255)),
    format='jpg',
)
face_preview = widgets.Image(
    value=img_to_value(np.full((10, 10, 3), dtype=np.uint8, fill_value=255)),
    format='jpg',
)
classifier_preview = widgets.Image(
    value=img_to_value(np.full((10, 10, 3), dtype=np.uint8, fill_value=255)),
    format='jpg',
)
covered_face_preview = widgets.Image(
    value=img_to_value(np.full((10, 10, 3), dtype=np.uint8, fill_value=255)),
    format='jpg',
)


def update_input_preview(img):
    if img is None:
        preview = get_image_with_text('No image selected', (125, 125, 125))
    elif img.shape[0] > 333:
        f = 333.0 / img.shape[0]
        preview = cv2.resize(img, None, fx=f, fy=f)
    else:
        preview = img.copy()

    input_image_preview.value = img_to_value(preview)

    
def update_face_preview(face, err_text=None, err_color=None):
    if face is None:
        err_text = 'Failed to find a face' if err_text is None else err_text
        err_color = (63, 0, 255) if err_color is None else err_color

        face_preview.value = img_to_value(get_image_with_text(err_text, err_color))
        classifier_preview.value = face_preview.value
        covered_face_preview.value = face_preview.value
    else:
        if face.shape[0] > 333:
            f = 333.0 / face.shape[0]
            face = cv2.resize(face, None, fx=f, fy=f)
        face_preview.value = img_to_value(face)


def update_classifier_preview(prob):
    GREEN = np.array([52, 203, 3])
    YELLOW = np.array([10, 182, 240])
    RED = np.array([63, 0, 255])
    
    text = f'Mask presence probability: {prob:.2f}'
    if prob < 0.5:
        raw_color = YELLOW * 2 * prob + RED * (1.0 - 2 * prob)
    else:
        raw_color = GREEN * 2 * (prob - 0.5) + YELLOW * (1.0 - 2 * (prob - 0.5))
        
    color = tuple(int(v) for v in raw_color)
    classifier_preview.value = img_to_value(get_image_with_text(text, color))


def update_covered_face_preview(covered):
    if covered is None:
        err_text = 'Please choose a person not wearing a mask'
        err_color = (63, 0, 255)
        covered_face_preview.value = img_to_value(get_image_with_text(err_text, err_color))
    else:
        covered_face_preview.value = img_to_value(covered)
    
    
def on_image_update(*args, **kwargs):
    img = get_input_image()
    update_input_preview(img)

    if img is None:
        update_face_preview(None, 'Please select an image first', (125, 125, 125))
        return

    boxes = face_boxes(img)
    if len(boxes) == 0:
        update_face_preview(None)
        return
    
    box = np.clip(np.round(np.array(boxes[0][:-1])).astype(int),
                  [0, 0] * 2,
                  [img.shape[1] - 1, img.shape[0] - 1] * 2)
    face = img[box[1]:box[3], box[0]:box[2]]
    update_face_preview(face)
    
    mask_prob = classify(image_bgr=face)
    update_classifier_preview(mask_prob)
    
    if mask_prob > 0.5:
        update_covered_face_preview(None)
        return
    
    covered_face_preview.value = img_to_value(get_image_with_text('Working...', (125, 125, 125)))

    facecover = cv2.imread(choice(FACECOVER_PATHS), cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.0
    covered = cover_face_with_mask(img, facecover)
    assert covered is not None
    
    update_covered_face_preview(covered)

on_image_update()    
upload.observe(on_image_update, names=['_counter'])

Preview:

In [ ]:
display(input_image_preview)

## Detected face

In [ ]:
display(face_preview)

## Classifier

In [ ]:
display(classifier_preview)

## Covered face with a mask

In [ ]:
display(covered_face_preview)

___